In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy as sp
import pandas as pd
import sklearn as sk
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.font_manager as fm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
sns.set(rc={'figure.figsize':(13.7,10.27)})
sns.set_style("whitegrid")
sns.set_color_codes()

In [2]:
from dask.distributed import Client
import dask.bag as db
import dask.dataframe as dd
import dask

from ast import literal_eval
from collections import MutableMapping
import dask.array as da
# import h5py
import os

from ast import literal_eval
from collections import MutableMapping

from geopy.geocoders import Nominatim
import reverse_geocode

In [3]:
client = Client(n_workers=8)

In [4]:
client

Client Scheduler: tcp://127.0.0.1:39537 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 33.65 GB


In [5]:
# Place your original data-sets like
# data/users.clean.json
# data/places.clean.json
# data/reviews.clean.json

In [6]:
folder_path = 'data/'
os.path.join(folder_path, "places", "*.csv")

'data/places/*.csv'

In [7]:
def drop_cols(obj):
    obj.pop('reviewerName')
    obj.pop('reviewText')
    obj.pop('unixReviewTime')
    obj.pop('reviewTime')
    
    return obj

In [8]:
def extract_restaurant_and_pub(obj):
    if obj["categories"]:
        for cat in obj["categories"]:
            if "Restaurant" in cat or "Pub" in cat or "Bar" in cat:
                return 1
    return 0

In [9]:
# # Store only restaurants reviews
# b = db.read_text(os.path.join(folder_path, "reviews.clean.json"), blocksize="64MiB")
# df = b.map(literal_eval).map(drop_cols).to_dataframe().repartition(npartitions=240)
# df["target"] = df.loc[:, ["categories"]].apply(extract_restaurant_and_pub, axis=1)
# df[(df["rating"].notnull()) & (df["target"]==1)].drop(["target"], axis=1).to_csv(os.path.join(folder_path, "reviews", "*.csv"), index=False)

In [10]:
# Load dataframes
reviews = dd.read_csv(os.path.join(folder_path, "reviews", "*.csv"), blocksize='64MiB')

In [11]:
reviews.head()

,rating,categories,gPlusPlaceId,gPlusUserId
0,4.0,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,100000032416892623125
1,5.0,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,100000032416892623125
2,5.0,['Barbecue Restaurant'],111623070919810985923,100000032416892623125
3,4.0,['Restaurant'],113854191152597312098,100000032416892623125
4,5.0,['Mexican Restaurant'],115827996910815192564,100000032416892623125


In [12]:
reviews.compute().shape

(4170168, 4)

In [13]:
# # Store places
# b = db.read_text(os.path.join(folder_path, "places.clean.json"), blocksize="64MiB")
# b.map(literal_eval).to_dataframe().repartition(npartitions=240).to_csv(os.path.join(folder_path, "places", "*.csv"), index=False)

In [26]:
# Load places
places = dd.read_csv(os.path.join(folder_path, "places", "*.csv"), blocksize='64MiB').dropna(subset=['gps','price', 'address']).drop(columns=['name','hours','phone','closed'])

In [27]:
places.head()

,price,address,gPlusPlaceId,gps
3,$$,"['5322 W 26th St', 'Sioux Falls, SD 57106']",100327153115986850675,"[43.529494, -96.792244]"
9,$$,"['17960 NW Evergreen Pkwy', 'Beaverton, OR 970...",118222137795476771294,"[45.535176, -122.862242]"
10,$$,"['3718 Wilmington Pike', 'Dayton, OH 45429']",106432060150136868000,"[39.692899, -84.136173]"
23,$$,"['3607 Outdoor Sportsman Pl', 'Kodak, TN 37764']",100184392614713668281,"[35.98598, -83.610598]"
31,$$$,"['242 Massachusetts Ave', 'Arlington, MA 02474']",110300304875024740707,"[42.406904, -71.143994]"


In [28]:
places.compute().shape

(400314, 4)

In [29]:
def drop_cols(obj):
    print(obj.items())
    obj.pop('jobs')
#     obj.pop('currentPlace')
    obj.pop('previousPlaces')
    obj.pop('education')
    return obj
def has_cordinates(obj):
    return obj['cordinates'] is not None

In [30]:
def get_cordinates(row):
#     print("getcordinates")
    currentPlace=row["currentPlace"]
    output = None
    try:
        cordinates = currentPlace[1][1:3]
        lon = int(cordinates[1])/10000000
        lat = int(cordinates[0])/10000000
        output = (lat, lon)
    except Exception as e:
        pass
    return output

In [31]:
def get_country(row):
    cordinates = row['Cordinates']
    country = None
    if cordinates:
        try:
            raw_location = reverse_geocode.search([cordinates])
            country = raw_location[0].get('country')
        except Exception as e:
            pass
    return country

In [32]:
# # Store Users
# b = db.read_text(os.path.join(folder_path, "users.clean.json"), blocksize="64MiB")
# df = b.map(eval).map(drop_cols).to_dataframe().dropna().repartition(npartitions=240)
# df['Cordinates'] = df.apply(get_cordinates, axis=1)
# df['Country'] = df.apply(get_country, axis=1)
# df = df.dropna(subset=["Cordinates"])
# df['latitude'] = df['Cordinates'].apply(lambda x : x[0])
# df['longitude'] = df['Cordinates'].apply(lambda x : x[1])
# df.to_csv(os.path.join(folder_path, "users", "*.csv"), index=False)

In [33]:
# Load Users
users = dd.read_csv(os.path.join(folder_path, "users", "*.csv"), blocksize='64MiB')

In [34]:
users.head()

,userName,currentPlace,gPlusUserId,Cordinates,Country,latitude,longitude
0,an lam,"['Thành phố Hồ Chí Minh, Việt Nam', [[], 10823...",100000010817154263736,"(10.823099, 106.629664)",Vietnam,10.823099,106.629664
1,HALİL TURGUT,"['Adana', [[], 370000000, 353213330, 1]]",100000013500285534661,"(37.0, 35.321333)",Turkey,37.000000,35.321333
2,Jacquelyn Dorris,"['Pomona, CA', [[], 340552270, -1177523050, 1]]",100000035085750632094,"(34.055227, -117.752305)",United States,34.055227,-117.752305
3,Aniello Prezioso,"['roma', [[], 418954660, 124823240, 1]]",100000067656171981860,"(41.895466, 12.482324)",Holy See (Vatican City State),41.895466,12.482324
4,Anne Mason,"['San Francisco, CA', [[], 377751960, -1224192...",100000106576186066497,"(37.775196, -122.419204)",United States,37.775196,-122.419204


In [35]:
users.compute().shape

(737639, 7)

In [37]:
# Create Join
reviews_users_join = reviews.merge(users, left_on="gPlusUserId", right_on="gPlusUserId")
reviews_users_join.head()

,rating,categories,gPlusPlaceId,gPlusUserId,userName,currentPlace,Cordinates,Country,latitude,longitude
0,5.0,['Thai Restaurant'],114263029861010899805,100038613745610076891,James Williams,"['Spokane, Wa', [[], 476587800, -1174260470, 1]]","(47.65878, -117.426047)",United States,47.658780,-117.426047
1,5.0,['Restaurant'],112027889074070004597,100049324721726350000,Mark Ram,"['Mississauga, ON', [[], 435890450, -796441200...","(43.589045, -79.64412)",Canada,43.589045,-79.644120
2,4.0,"['Italian Restaurant', 'European Restaurant', ...",109297670713770542826,100058232515318296419,Martin Jørgensen,"['Frederiksberg, Denmark', [[], 556770690, 125...","(55.677069, 12.513321)",Denmark,55.677069,12.513321
3,4.0,"['Latin American Restaurant', 'Mexican Restaur...",110719164957604789311,100068477230676623614,Greg Urick,"['Carson-Tahoe, NV', [[], 391272560, -12002029...","(39.127256, -120.020296)",United States,39.127256,-120.020296
4,4.0,"['Restaurant', 'Travel']",105539154444992983581,100080606446951932523,RUSSELL CHERRY,"['Athens, GA 30606', [[], 339448440, -83432338...","(33.944844, -83.432338)",United States,33.944844,-83.432338


In [38]:
reviews_users_join.compute().shape

(1026054, 10)

In [39]:
reviews_users_places_join = reviews_users_join.merge(places, left_on="gPlusPlaceId", right_on="gPlusPlaceId")
reviews_users_places_join.head()

,rating,categories,gPlusPlaceId,gPlusUserId,userName,currentPlace,Cordinates,Country,latitude,longitude,price,address,gps
0,1.0,"['Thai Restaurant', 'Asian Restaurant', 'South...",100986930027386372345,100817491644930023211,Daniel Lott,"['Southcentral Louisiana', [[], 300793120, -90...","(30.079312, -90.567878)",United States,30.079312,-90.567878,$,"['4335 Perkins Rd', 'Baton Rouge, LA 70808']","[30.417639, -91.145692]"
1,4.0,"['Thai Restaurant', 'Asian Restaurant', 'South...",100986930027386372345,108477879612577186121,Randall Godso,"['Baton Rouge, LA', [[], 304507460, -911545510...","(30.450746, -91.154551)",United States,30.450746,-91.154551,$,"['4335 Perkins Rd', 'Baton Rouge, LA 70808']","[30.417639, -91.145692]"
2,1.0,"['Thai Restaurant', 'Asian Restaurant', 'South...",100986930027386372345,107244184518329135590,Bradley Lynch,"['Baton Rouge, LA', [[], 304582830, -911403200...","(30.458283, -91.14032)",United States,30.458283,-91.140320,$,"['4335 Perkins Rd', 'Baton Rouge, LA 70808']","[30.417639, -91.145692]"
3,5.0,"['Thai Restaurant', 'Asian Restaurant', 'South...",100986930027386372345,102984966038548477374,Stephen Hawkins,"['New Orleans', [[], 299510660, -900715320, 1]]","(29.951066, -90.071532)",United States,29.951066,-90.071532,$,"['4335 Perkins Rd', 'Baton Rouge, LA 70808']","[30.417639, -91.145692]"
4,4.0,"['Steak House', 'European Restaurant', 'Italia...",114437509036704314647,102397052073935758476,Kelly Fee,"['San Francisco, CA', [[], 377749300, -1224194...","(37.77493, -122.419416)",United States,37.774930,-122.419416,$,"['5700 S Cicero Ave', 'Chicago, IL 60638']","[41.788477, -87.742125]"


In [40]:
reviews_users_places_join.compute().shape

(476390, 13)

In [44]:
reviews_users_places_join.to_csv(os.path.join(folder_path, "join", "*.csv"), index=False)

['/home/ej/github/cmpe256-team4/data/join/000.csv',
 '/home/ej/github/cmpe256-team4/data/join/001.csv',
 '/home/ej/github/cmpe256-team4/data/join/002.csv',
 '/home/ej/github/cmpe256-team4/data/join/003.csv',
 '/home/ej/github/cmpe256-team4/data/join/004.csv',
 '/home/ej/github/cmpe256-team4/data/join/005.csv',
 '/home/ej/github/cmpe256-team4/data/join/006.csv',
 '/home/ej/github/cmpe256-team4/data/join/007.csv',
 '/home/ej/github/cmpe256-team4/data/join/008.csv',
 '/home/ej/github/cmpe256-team4/data/join/009.csv',
 '/home/ej/github/cmpe256-team4/data/join/010.csv',
 '/home/ej/github/cmpe256-team4/data/join/011.csv',
 '/home/ej/github/cmpe256-team4/data/join/012.csv',
 '/home/ej/github/cmpe256-team4/data/join/013.csv',
 '/home/ej/github/cmpe256-team4/data/join/014.csv',
 '/home/ej/github/cmpe256-team4/data/join/015.csv',
 '/home/ej/github/cmpe256-team4/data/join/016.csv',
 '/home/ej/github/cmpe256-team4/data/join/017.csv',
 '/home/ej/github/cmpe256-team4/data/join/018.csv',
 '/home/ej/g

In [47]:
join_data = dd.read_csv(os.path.join(folder_path, "join", "*.csv"), blocksize='64MiB')
join_data.head()

,rating,categories,gPlusPlaceId,gPlusUserId,userName,currentPlace,Cordinates,Country,latitude,longitude,price,address,gps
0,1.0,"['Thai Restaurant', 'Asian Restaurant', 'South...",100986930027386372345,100817491644930023211,Daniel Lott,"['Southcentral Louisiana', [[], 300793120, -90...","(30.079312, -90.567878)",United States,30.079312,-90.567878,$,"['4335 Perkins Rd', 'Baton Rouge, LA 70808']","[30.417639, -91.145692]"
1,4.0,"['Thai Restaurant', 'Asian Restaurant', 'South...",100986930027386372345,108477879612577186121,Randall Godso,"['Baton Rouge, LA', [[], 304507460, -911545510...","(30.450746, -91.154551)",United States,30.450746,-91.154551,$,"['4335 Perkins Rd', 'Baton Rouge, LA 70808']","[30.417639, -91.145692]"
2,1.0,"['Thai Restaurant', 'Asian Restaurant', 'South...",100986930027386372345,107244184518329135590,Bradley Lynch,"['Baton Rouge, LA', [[], 304582830, -911403200...","(30.458283, -91.14032)",United States,30.458283,-91.140320,$,"['4335 Perkins Rd', 'Baton Rouge, LA 70808']","[30.417639, -91.145692]"
3,5.0,"['Thai Restaurant', 'Asian Restaurant', 'South...",100986930027386372345,102984966038548477374,Stephen Hawkins,"['New Orleans', [[], 299510660, -900715320, 1]]","(29.951066, -90.071532)",United States,29.951066,-90.071532,$,"['4335 Perkins Rd', 'Baton Rouge, LA 70808']","[30.417639, -91.145692]"
4,4.0,"['Steak House', 'European Restaurant', 'Italia...",114437509036704314647,102397052073935758476,Kelly Fee,"['San Francisco, CA', [[], 377749300, -1224194...","(37.77493, -122.419416)",United States,37.774930,-122.419416,$,"['5700 S Cicero Ave', 'Chicago, IL 60638']","[41.788477, -87.742125]"


In [50]:
join_data['gps'].compute().isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
2187    False
2188    False
2189    False
2190    False
2191    False
Name: gps, Length: 476390, dtype: bool

In [48]:
join_data.compute().shape

(476390, 13)